<a href="https://colab.research.google.com/github/Sarthak-125/PINN_2/blob/main/Berger_Equation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the required libraries

In [1]:
import numpy as np
import tensorflow as tf
import random

Function for generating random arrays

In [2]:
def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res

Generating points of computation

In [3]:
n_c = 5000 #collocation points
n_b = 1000 #boundary points on each boundary
n_i = 5000 #intial points

#collocation point generation
x_c = Rand(0, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = tf.cast(x_c ,dtype='float32')
t_c = tf.cast(t_c ,dtype='float32')

#boundary point generation
x_lb = np.zeros((n_b,), dtype='float32')
t_lb = Rand(0, 1, n_b)
x_rb = np.ones((n_b,), dtype='float32')
t_rb = Rand(0, 1, n_b)
x_lb = tf.cast(x_lb ,dtype='float32')
t_lb = tf.cast(t_lb ,dtype='float32')
x_rb = tf.cast(x_rb ,dtype='float32')
t_rb = tf.cast(t_rb ,dtype='float32')

#intialisation point generation
x_i =  Rand(0, 1, n_i)
t_i = np.zeros((n_i,), dtype='float32')
x_i = tf.cast(x_i, dtype=tf.float32)
t_i = tf.cast(t_i, dtype=tf.float32)

BC and Initial Condition
*   u(x=0,t) = u_o
*   u(x=1, t) = u_1
*   u(x, t=0) = g(x)

In [4]:
u_0 = float(input("Boundary value at x=0  :"));
u_1 = float(input("Boundary value at x=1   :"))
#inputing the initial function at t=0 let it be
g = eval(input("Input function at t=0  :"))

Boundary value at x=00
Boundary value at x=11
Input function at t=0 :np.sin


Entering the value of nu

In [5]:
nu = float(input("Enter the value of nu   :"))

Enter the value of nu   :0.003183


In [6]:
#creating output for boundary point compatible for training
u_lb = u_0*np.ones((n_b,), dtype='float32')
u_lb = tf.cast(u_lb, dtype=tf.float32)
u_rb = u_1*np.ones((n_b,), dtype='float32')
u_rb = tf.cast(u_rb, dtype=tf.float32)

#creating initialising points outputs
u_i = g(x_i)
u_i = tf.cast(u_i, dtype='float32')

Creating input for the loss function

In [7]:
x_loss = tf.concat((x_c, x_lb, x_rb, x_i), axis=0)
t_loss = tf.concat((t_c, t_lb, t_rb, t_i), axis=0)

Creating input for the training

In [8]:
x_train = tf.concat((x_lb, x_rb, x_i), axis=0)
t_train = tf.concat((t_lb, t_rb, t_i), axis=0)
u_train = tf.concat((u_lb, u_rb, u_i), axis=0)

Creating the architecture

In [10]:
x_in = tf.keras.layers.Input(shape=(1), name="Position")
t_in = tf.keras.layers.Input(shape=(1), name="Time")
IN = tf.keras.layers.Concatenate(axis=-1, name="Input")([x_in, t_in])
dense1=(tf.keras.layers.Dense(50,activation='sigmoid'))(IN)
dense2=(tf.keras.layers.Dense(50,activation='sigmoid'))(dense1)
dense3=(tf.keras.layers.Dense(50,activation='sigmoid'))(dense2)
dense4=(tf.keras.layers.Dense(50,activation='sigmoid'))(dense3)
dense5=(tf.keras.layers.Dense(50,activation='sigmoid'))(dense4)
output=(tf.keras.layers.Dense(1,activation='sigmoid'))(dense5)
pinn2 = tf.keras.Model(inputs=[x_in,t_in], outputs=output, name="pinn2")

Creating the loss function for the model

In [11]:
def custom_loss_2(arch, x, t):
  def loss(u_train, u_pred):
    with tf.GradientTape(persistent=True) as t1:
      t1.watch(x)
      t1.watch(t)
      with tf.GradientTape() as t2:
        t2.watch(x)
        u = arch([x, t])
        u_x = t2.gradient(u, x)
      u_t = t1.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
    u_pred = tf.cast(u_pred, dtype='float32')
    u_train = tf.cast(u_train, dtype='float32')      
    f = u_t + u*u_x - (nu)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    loss_mse = tf.reduce_mean(tf.square(u_pred-u_train))
    return loss_res + loss_mse
  return loss

Compiling and fitting the model

In [12]:
loss_fn = custom_loss_2(pinn2, x_loss, t_loss)
pinn2.compile(optimizer='adam', loss=loss_fn, metrics=['mse', 'mae'])
pinn2.fit([x_train, t_train], u_train, batch_size=32, epochs=10)

Epoch 1/10
219/219 [==============================] - 442s 2s/step - loss: 0.1207 - mse: 0.1207 - mae: 0.3015
Epoch 2/10
219/219 [==============================] - 426s 2s/step - loss: 0.1075 - mse: 0.1062 - mae: 0.2789
Epoch 3/10
219/219 [==============================] - 424s 2s/step - loss: 0.0354 - mse: 0.0233 - mae: 0.1096
Epoch 4/10
219/219 [==============================] - 424s 2s/step - loss: 0.0359 - mse: 0.0236 - mae: 0.1110
Epoch 5/10
219/219 [==============================] - 425s 2s/step - loss: 0.0348 - mse: 0.0226 - mae: 0.1077
Epoch 6/10
219/219 [==============================] - 425s 2s/step - loss: 0.0354 - mse: 0.0229 - mae: 0.1089
Epoch 7/10
219/219 [==============================] - 431s 2s/step - loss: 0.0344 - mse: 0.0223 - mae: 0.1079
Epoch 8/10
219/219 [==============================] - 440s 2s/step - loss: 0.0355 - mse: 0.0230 - mae: 0.1097
Epoch 9/10
219/219 [==============================] - 433s 2s/step - loss: 0.0346 - mse: 0.0224 - mae: 0.1077
Epoch 10/1

In [35]:
x_test = []
t_test = []
for i in range(10):
  x_test.append(x_loss[random.randint(0,n_c+n_i)])
  t_test.append(t_loss[random.randint(0,n_c+n_i)])

x_test = tf.cast(x_test, dtype='float32')
t_test = tf.cast(t_test, dtype='float32')

In [36]:
print(x_test)

tf.Tensor(
[0.713813   0.8751826  0.         0.42572826 0.8582346  0.23515816
 0.5294696  0.6035047  0.5037263  0.49669734], shape=(10,), dtype=float32)


In [37]:
u = pinn2([x_test,t_test])

for i in range(10):
  tf.print("u at x=", x_test[i], "and at time t = ", t_test[i], "is =" , u[i]);

u at x= 0.713813 and at time t =  0 is = [0.631791472]
u at x= 0.875182629 and at time t =  0.639514387 is = [0.576492369]
u at x= 0 and at time t =  0.426178247 is = [0.151594549]
u at x= 0.425728261 and at time t =  0.621804 is = [0.258436799]
u at x= 0.858234584 and at time t =  0 is = [0.699558675]
u at x= 0.23515816 and at time t =  0.647011578 is = [0.182640523]
u at x= 0.529469609 and at time t =  0.455301464 is = [0.365153074]
u at x= 0.603504717 and at time t =  0.283002555 is = [0.47646606]
u at x= 0.503726304 and at time t =  0 is = [0.48457846]
u at x= 0.496697336 and at time t =  0.671564579 is = [0.286758155]
